In [1]:
import hashlib

In [2]:
def sha256double(byts):
    '''Purpose SHA256 twice to the bytes.'''
    return hashlib.sha256(hashlib.sha256(byts).digest()).digest()

In [3]:
class MerkleTree:
    def __init__(self):
        self.txid_list = []
        self.merkle_root = ''

    def make_merkle_tree(self, txid_list, is_sorted=True):
        '''Compute merkle root for the given list of TXIDs.'''
        if len(txid_list) == 0:
            raise ValueError("empty list")

        # Sort TXIDs (strings) in lexicographical order if not ordered
        if is_sorted:
            self.txid_list = txid_list
        else:
            self.txid_list = sorted(txid_list)

        # Compute merkle root row by row
        hashes = [s.encode() for s in self.txid_list]
        while len(hashes) > 1:
            if len(hashes) & 1:  # Number of hashes is odd
                hashes.append(hashes[-1])
            next_hashes = []
            for i in range(0, len(hashes), 2):
                next_hashes.append(hashes[i] + hashes[i+1])
            hashes = [sha256double(s) for s in next_hashes]  # Hashes are bytes, not texts

        self.merkle_root = hashes[0]
        return self.merkle_root  # 32 bytes

In [4]:
mt = MerkleTree()
mt.make_merkle_tree(["aaa", "bbb"])

b'?\x08\\8\x1c\x0b\xa2egr\x03\x7f\xb9/3\xce{\x16\xafg\x18?\x1ds\xb0\x8c\xa0\xe4\x19\xfaA\x0f'

## 2 生成账户

In [29]:
from ecdsa import SigningKey, VerifyingKey, SECP256k1
import base58

class Account:
    def __init__(self, prikey=''):
        '''
        Create a Bitcoin account using the given private key.
        If private key not provided, generate a new one randomly.
        '''
        if prikey:
            self.private_key = prikey
            self.public_key = gen_pubkey_from_prikey(prikey)
        else:
            self.private_key, self.public_key = self.gen_keypair()
        
        self.public_key_hash = self.gen_public_key_hash(self.public_key)
        self.address = self.gen_address(self.public_key_hash)

    @staticmethod
    def gen_keypair():
        '''Generate a new pair of keys using SECP256k1 ECDSA.'''
        sk = SigningKey.generate(curve=SECP256k1)
        vk = sk.verifying_key
        return sk.to_string(), vk.to_string()
        
    def gen_pubkey_from_prikey(prikey):
        '''Compute public key from the given private key.'''
        sk = SigningKey.from_string(prikey, curve=SECP256k1)
        vk = sk.verifying_key
        return vk.to_string()

    @staticmethod
    def gen_public_key_hash(pubkey):
        '''Compute public key hash.'''
        temp = hashlib.sha256(pubkey).digest()
        h = hashlib.new('ripemd160')
        h.update(temp)
        return h.digest()

    @staticmethod
    def gen_address(pubkeyhash):
        '''Compute base58check encoded address.'''
        
        # Base58check encode
        version_byte = b'\x00'
        return base58.b58encode_check(version_byte + pubkeyhash)

In [30]:
account = Account()
account.address

b'14Ry7ULerUfMZ9oWUktyXoKd3vahe1C9qv'

In [6]:
testpubkey = bytes.fromhex("0250863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b2352")

h = hashlib.new('ripemd160')
h.update(hashlib.sha256(testpubkey).digest())
public_key_hash = h.digest()

# Base58check encode
address = base58.b58encode_check(b'\x00' + h.digest())
address

b'1PMycacnJaSqwwJqjawXBErnLsZ7RkXUAs'

In [7]:
h.digest()

b'\xf5JXQ\xe97+\x87\x81\n\x8e`\xcd\xd2\xe7\xcf\xd8\x0bn1'

## 3 消息签名和验证

## 4 生成自定义交易

## 5 打包成区块，组织成链